Code to clean JSON and csv inputs for better outputs

In [14]:
# necessary imports

import pandas as pd
import numpy as np
import csv
import json
import re
from difflib import get_close_matches # for fuzzy matching
from collections import defaultdict
import instaloader

In [5]:
# Read the cleaned restaurants data
restaurants_df = pd.read_csv('../outputs/cleaned_restaurants.csv')

# Create priority for A/B/C grades
grade_priority = {'A': 1, 'B': 2, 'C': 3}
restaurants_df['grade_priority'] = restaurants_df['GRADE'].map(grade_priority).fillna(4)

# Sort by DBA and grade priority
restaurants_df = restaurants_df.sort_values(by=['DBA', 'grade_priority'])

# Remove duplicates, keeping first valid grade
restaurants_df = restaurants_df.drop_duplicates(subset=['DBA'], keep='first')

# If grade wasn't A/B/C, set to blank
restaurants_df.loc[~restaurants_df['GRADE'].isin(['A', 'B', 'C']), 'GRADE'] = ''

# Read the borough cuisines data
borough_df = pd.read_csv('../outputs/nyc_borough_cuisines.csv')

# Convert to lowercase for case-insensitive matching
restaurants_df['DBA_lower'] = restaurants_df['DBA'].str.lower()
borough_df['name_lower'] = borough_df['name'].str.lower()

# Merge with case-insensitive matching
merged_df = pd.merge(
    restaurants_df[['DBA', 'BORO', 'GRADE', 'DBA_lower']],
    borough_df[['name', 'rating', 'review_count', 'categories_list', 'name_lower']],
    left_on='DBA_lower',
    right_on='name_lower',
    how='inner'
).drop_duplicates(subset=['DBA_lower'])

# Rename columns and select final columns
merged_df = merged_df.rename(columns={
    'DBA': 'name',
    'BORO': 'borough'
})[['name', 'borough', 'rating', 'review_count', 'categories_list', 'GRADE']]

# Save to final merged file
merged_df.to_csv('../outputs/merged_restaurants_final.csv', index=False, encoding='utf-8')

In [8]:
# get restaurant names from inspection data
try:
        df = pd.read_csv("../outputs/merged_restaurant_data.csv")

        # Extract the 'name' column and add "NYC restaurant"
        restaurant_names = df['name'].tolist()  # Convert to a list
        formatted_names = [f"{name} NYC restaurant" for name in restaurant_names]


        # Write names to text file
        with open("../outputs/restaurant_names.txt", 'w', encoding='utf-8') as txtfile: # Use encoding to support non-ascii
            for name in formatted_names:
                txtfile.write(name + '\n') # newline after each name

        print(f"Restaurant names saved to {"../outputs/restaurant_names.txt"}")

except FileNotFoundError:
    print(f"Error: Input CSV file '{"../outputs/merged_restaurant_data.csv"}' not found.")
except KeyError as e:
    print(f"Error: Column '{e}' not found in the CSV. Check the header.")
except Exception as e:
    print(f"An error occurred: {e}")

Restaurant names saved to ../outputs/restaurant_names.txt


In [10]:
# add instagram data restaurant names

with open("../outputs/IG_posts.json", "r", encoding="utf-8") as f:
    insta_data = json.load(f)
    

restaurant_names = set()

for entry in insta_data:
    caption = entry.get("caption", "").lower()
    
    # Extract restaurant names from caption
    possible_names = re.findall(r'@([a-zA-Z0-9_]+)', caption)
    restaurant_names.update(possible_names)

restaurant_names = list(restaurant_names)

# put into file

with open("../outputs/restaurant_names.json", "w") as file:
    
    json.dump(restaurant_names, file, indent=4)
    
    print("Data has been written to file")
    


Data has been written to file


In [14]:
# create nodes for Gephi

"""Creates a Gephi-compatible nodes CSV from the restaurant data."""
try:
    df = pd.read_csv("../outputs/merged_restaurant_data.csv")

    # Select desired columns and rename "DBA" to "Label" for Gephi compatibility
    nodes_df = df[["name", "BOROUGH", "categories_list", "SCORE", "GRADE"]].copy()  # Create a copy to avoid warnings

    nodes_df.rename(columns={"name": "Label"}, inplace=True) #Rename here

    # Add an "Id" column (essential for Gephi) - use index as ID
    nodes_df.insert(0, 'Id', range(len(nodes_df)))  # Insert at beginning


    # Ensure 'SCORE' is numeric and handle errors gracefully
    nodes_df['SCORE'] = pd.to_numeric(nodes_df['SCORE'], errors='coerce') # Convert to numeric, invalid values become NaN


    # Handle potential empty or NaN values (replace with empty string to avoid Gephi import issues)
    nodes_df.fillna('', inplace=True) #Important to use fillna after to_numeric.  Otherwise you replace valid numbers


    # Write nodes to CSV
    nodes_df.to_csv("../outputs/nodes.csv", index=False, encoding="utf-8")

    print(f"Nodes CSV saved to {"../outputs/nodes.csv"}")


except FileNotFoundError:
    print(f"Error: Input CSV file not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Nodes CSV saved to ../outputs/nodes.csv


In [17]:
# create edges for gephi

df = pd.read_csv("../outputs/nodes.csv")
edges = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        if df.iloc[i]['BOROUGH'] == df.iloc[j]['BOROUGH']:
            edges.append([df.iloc[i]['Id'], df.iloc[j]['Id'], "Undirected", 1])  # Create edge


edges_df = pd.DataFrame(edges, columns=["Source", "Target", "Type", "Weight"])
edges_df.to_csv("../outputs/boro_edges.csv", index=False)

In [23]:
# filter google places reviews

with open("../outputs/review_aggregated.json", "r", encoding="utf-8") as f:
    review_data = json.load(f)
    
filtered_reviews = []

for entry in review_data:
    
    updates = entry.get("updatesFromCustomers", {})
    posted_by = updates.get("postedBy", {}) if updates else {}
    
    filtered_review = {
        "city": entry.get("city"),
        "category": entry.get("categoryName"),
        "title": entry.get("title"),
        "score" : entry.get("totalScore"),
        "review": updates.get("text") if updates else None,
        "name" : posted_by.get("name") if posted_by else None,
        "links" : [media.get("link") for media in updates.get("media", [])] if updates else []
    }
    
    filtered_reviews.append(filtered_review)

with open("../outputs/filtered_reviews.json", "w") as file:
    json.dump(filtered_reviews, file, indent=4)
    
    print("Data has been written to file")

Data has been written to file


In [24]:
# only keep those review names which we have active information for
with open("../outputs/restaurant_names.txt", "r", encoding="utf-8") as f:
    restaurant_names = [line.strip().replace(" NYC restaurant", "") for line in f.readlines()]
    
with open("../outputs/filtered_reviews.json", "r", encoding="utf-8") as f:
    filtered_data = json.load(f)
    
filtered_data = [entry for entry in filtered_data if entry.get("title") and get_close_matches(entry.get("title"), restaurant_names, n=1, cutoff=0.6)]

with open("../outputs/filtered_reviews.json", "w", encoding = "utf-8") as file:
    json.dump(filtered_data, file, indent=4, ensure_ascii=False)

In [25]:
# dump those restaurants into a .txt file 

with open("../outputs/filtered_reviews.txt", "w", encoding="utf-8") as f:
    for entry in filtered_data:
        f.write(entry.get("title") + "\n")

In [ ]:
# remove blank fields (where restaurant_name = "") in the instagram posts

# this is to be run after extract_restaurant_names.py
# Load data from the JSON file
with open("../outputs/IG_posts.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Remove entries where restaurantName is empty
filtered_data = [entry for entry in data if entry.get("restaurantName", "").strip() != ""]

# Save the filtered data back to the same file (or change file_path to save as a new file)
with open("../outputs/IG_posts.json", "w", encoding="utf-8") as file:
    json.dump(filtered_data, file, indent=2, ensure_ascii=False)

print(f"Filtered data saved. {len(filtered_data)} entries remaining.")


Filtered and decoded data saved. 3779 entries remaining.


In [ ]:
# Grab popular influencers using instaloader

# Initialize Instaloader
L = instaloader.Instaloader()

# Load posts from the JSON file

with open('../outputs/IG_posts.json', 'r') as file:
    posts = json.load(file)

influencers = {}

# Process each post
for post in posts:
    username = post.get('influencerUsername')
    restaurant_name = post.get('restaurantName')
    
    # Avoid processing duplicates
    if username in influencers:
        continue
    
    print(f"Checking influencer: {username}")
    try:
        profile = instaloader.Profile.from_username(L.context, username)
        follower_count = profile.followers
    except Exception as e:
        print(f"Error retrieving profile for {username}: {e}")
        continue
    
    print(f"{username} has {follower_count} followers.")
    if follower_count > follower_threshold:
        influencers[username] = {
            "influencerUsername": username,
            "restaurantName": restaurant_name,
            "followers": follower_count  # Optional: add the followers count for reference.
        }
        print(f"Added {username} to the list of NYC food influencers.")
    else:
        print(f"Skipped {username} as follower count is below threshold.")

# Write the qualifying influencers to the output JSON file

with open("../outputs/nyc_food_influencers.json", 'w') as out_file:
    json.dump(list(influencers.values()), out_file, indent=2, ensure_ascii=False)
print(f"\nSaved {len(influencers)} influencers to {output_file}")




In [10]:
# deal with unicode error

with open("../outputs/IG_posts.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Save it back using actual symbols (not \u escape sequences)
with open("../outputs/IG_posts.json", "w", encoding="utf-8") as file:
    json.dump(data, file, indent=2, ensure_ascii=False)

print("All Unicode characters have been converted to their respective symbols.")


All Unicode characters have been converted to their respective symbols.


In [12]:
# auxillary function to reconcile if same name -> assign ratingAvailable: true
# run periodically as I was going through IG_posts.json by hand

with open("../outputs/IG_posts.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Step 1: Identify restaurantNames where any entry has ratingAvailable == true
restaurants_with_rating = set(
    entry["restaurantName"]
    for entry in data
    if entry.get("ratingAvailable") is True
)

# Step 2: Set ratingAvailable = True for all matching restaurantNames where it's missing
for entry in data:
    if (
        entry.get("restaurantName") in restaurants_with_rating and
        "ratingAvailable" not in entry
    ):
        entry["ratingAvailable"] = True

# Step 3: Save back to the file
with open("../outputs/IG_posts.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ All missing 'ratingAvailable' fields filled where appropriate.")

✅ All missing 'ratingAvailable' fields filled where appropriate.
